<a href="https://colab.research.google.com/github/simran-dk/777-Term-Project-Team9/blob/main/777_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=fe9cac3571b22a77a30c5d7ecf1e7f61a9cf2ad5f1c3346bed06a7cd90a92a4f
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
# Mount data from drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, expr, when, udf, explode_outer, collect_list
from pyspark.sql.types import IntegerType, StringType, StructType, StructField
from datetime import datetime

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("Process JSON Files") \
    .getOrCreate()

# Read all JSON files from a directory
json_directory_path = "/content/drive/My Drive/raw_data"
df = spark.read.option("multiline", "true").json(json_directory_path)

def clean_major(degree_list):
    if not degree_list:
        return None
    major = degree_list.pop(-1)
    if any(char.isnumeric() for char in major):
        return None
    return major

def convert_to_std_date(date_str):
    if date_str is None:
        return None

    try:
        date_obj = datetime.strptime(date_str, "%b %Y")
    except ValueError:
        try:
            # Handle the case where only the year is provided
            date_obj = datetime.strptime(date_str, "%Y")
        except ValueError:
            return None

    standard_date = date_obj.strftime("%Y-%m-%d")
    return standard_date


def convert_to_number(s):
    if s is None:
        return None
    try:
        s = s.strip().replace(',', '')
        if 'K' in s:
            s = s.replace('K', '')
            return int(float(s) * 1000)
        elif '+' in s:
            s = s.replace('+', '')
            return int(s)
    except ValueError:
        return None
    return int(s)

def clean_degree_level(degree: str) -> str:
    if not degree:
        return None
    degree = degree.strip().lower()
    if any([d in degree for d in ["master of science", "master", "ms", "m.s", "m.a.", "ma"]]):
        return "master"
    elif any([d in degree for d in ['doctor', 'phd']]):
        return "PhD"
    elif any([d in degree for d in ['bs', 'b.s', 'bachelor', 'b.a', 'ba']]):
        return "bachelor"
    elif any([d in degree for d in ["associate's", "associate"]]):
      return "associate"
    return degree


clean_major_udf = udf(clean_major)
clean_degree_level_udf = udf(clean_degree_level, StringType())
convert_to_std_date_udf = udf(convert_to_std_date, StringType())
convert_to_number_udf = udf(convert_to_number, IntegerType())

df_degree_info = df.withColumn("degree_info", explode(df.education.degree))
df_majors = df_degree_info \
    .withColumn("degree_name", col("degree_info").getItem(0)[0]) \
    .withColumn("major", col("degree_info").getItem(0)[1]) \
    .groupBy("id") \
    .agg(collect_list("major").alias("majors"), collect_list("degree_name").alias("degree_levels"))


# Extract job titles and company names
df_positions = df.withColumn("experience_info", explode(col("experiences"))) \
        .withColumn("positions", explode(col("experience_info.positions"))) \
        .withColumn("job_title", col("positions.job_title")) \
        .withColumn("company_name", col("experience_info.company_name")) \
        .groupBy("id") \
        .agg(collect_list("job_title").alias("positions"), collect_list("company_name").alias("companies"))

df_activities=df.withColumn("followers", convert_to_number_udf(col("activity.followers"))) \
               .withColumn("connections", convert_to_number_udf(col("activity.connections")))

# Join the major, degree level, positions, and company information
df_1 = df_majors.join(df_positions, on="id", how="inner") \
               .select("id", "majors", "degree_levels", "positions", "companies") \

df_final = df_1.join(df_activities, on="id", how="inner") \
               .select("id", "majors", "degree_levels", "positions", "companies","followers","connections")

df_final.show(truncate=False)
# Stop SparkSession
spark.stop()
#.withColumn("Start date of Job", convert_to_std_date_udf(df.experiences.positions.start[0]))\
#.drop(*["activity", "last_generated_at", "education","experiences","skills"])

+------------------------------------+------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------